<a href="https://colab.research.google.com/github/peterchang0414/hmm-jax/blob/main/fixed_lag_smoother.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flax

     |████████████████████████████████| 184 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 12.5 MB/s 
     |████████████████████████████████| 72 kB 604 kB/s 


In [2]:
# Inference and learning code for Hidden Markov Models using discrete observations.
# Has Jax version of each function. For the Numpy version, please see hmm_numpy_lib.py
# The Jax version of inference (not learning)
# has been upstreamed to https://github.com/deepmind/distrax/blob/master/distrax/_src/utils/hmm.py.
# This version is kept for historical purposes.
# Author: Gerardo Duran-Martin (@gerdm), Aleyna Kara (@karalleyna), Kevin Murphy (@murphyk)

from jax import lax
from jax.scipy.special import logit
from functools import partial

import jax.numpy as jnp
from scipy.special import softmax
from jax import vmap
from dataclasses import dataclass

import jax
import itertools
from jax import jit
from jax.nn import softmax
from jax.random import PRNGKey, split, normal
from jax.random import split, randint, PRNGKey, normal, permutation

import flax

'''
Hidden Markov Model class used in jax implementations of inference algorithms.
The functions of optimizers expect that the type of its parameters 
is pytree. So, they cannot work on a vanilla dataclass. To see more:
                https://github.com/google/jax/issues/2371
Since the flax.dataclass is registered pytree beforehand, it facilitates to use
jit, vmap and optimizers on the hidden markov model.
'''


@flax.struct.dataclass
class HMMJax:
    trans_mat: jnp.array  # A : (n_states, n_states)
    obs_mat: jnp.array  # B : (n_states, n_obs)
    init_dist: jnp.array  # pi : (n_states)


def normalize(u, axis=0, eps=1e-15):
    '''
    Normalizes the values within the axis in a way that they sum up to 1.
    Parameters
    ----------
    u : array
    axis : int
    eps : float
        Threshold for the alpha values
    Returns
    -------
    * array
        Normalized version of the given matrix
    * array(seq_len, n_hidden) :
        The values of the normalizer
    '''
    u = jnp.where(u == 0, 0, jnp.where(u < eps, eps, u))
    c = u.sum(axis=axis, keepdims=True)
    c = jnp.where(c == 0, 1, c)
    return u / c, c


##############################
# Inference

def hmm_forwards_jax(params, obs_seq, length=None):
    '''
    Calculates a belief state
    Parameters
    ----------
    params : HMMJax
        Hidden Markov Model
    obs_seq: array(seq_len)
        History of observable events
    Returns
    -------
    * float
        The loglikelihood giving log(p(x|model))
    * array(seq_len, n_hidden) :
        All alpha values found for each sample
    '''
    seq_len = len(obs_seq)

    if length is None:
        length = seq_len

    trans_mat, obs_mat, init_dist = params.trans_mat, params.obs_mat, params.init_dist

    trans_mat = jnp.array(trans_mat)
    obs_mat = jnp.array(obs_mat)
    init_dist = jnp.array(init_dist)

    n_states, n_obs = obs_mat.shape

    def scan_fn(carry, t):
        (alpha_prev, log_ll_prev) = carry
        alpha_n = jnp.where(t < length,
                            obs_mat[:, obs_seq[t]] * (alpha_prev[:, None] * trans_mat).sum(axis=0),
                            jnp.zeros_like(alpha_prev))

        alpha_n, cn = normalize(alpha_n)
        carry = (alpha_n, jnp.log(cn) + log_ll_prev)

        return carry, alpha_n

    # initial belief state
    alpha_0, c0 = normalize(init_dist * obs_mat[:, obs_seq[0]])

    # setup scan loop
    init_state = (alpha_0, jnp.log(c0))
    ts = jnp.arange(1, seq_len)
    carry, alpha_hist = lax.scan(scan_fn, init_state, ts)

    # post-process
    alpha_hist = jnp.vstack([alpha_0.reshape(1, n_states), alpha_hist])
    (alpha_final, log_ll) = carry
    return log_ll, alpha_hist


@jit
def hmm_loglikelihood_jax(params, observations, lens):
    '''
    Finds the loglikelihood of each observation sequence parallel using vmap.
    Parameters
    ----------
    params : HMMJax
        Hidden Markov Model
    observations: array(N, seq_len)
        Batch of observation sequences
    lens : array(N, seq_len)
        Consists of the valid length of each observation sequence
    Returns
    -------
    * array(N, seq_len)
        Consists of the loglikelihood of each observation sequence
    '''

    def forward_(params, x, length):
        return hmm_forwards_jax(params, x, length)[0]

    return vmap(forward_, in_axes=(None, 0, 0))(params, observations, lens)


def hmm_backwards_jax(params, obs_seq, length=None):
    '''
    Computes the backwards probabilities
    Parameters
    ----------
    params : HMMJax
        Hidden Markov Model
    obs_seq: array(seq_len,)
        History of observable events
    length : array(seq_len,)
        The valid length of the observation sequence
    Returns
    -------
    * array(seq_len, n_states)
       Beta values
    '''
    seq_len = len(obs_seq)

    if length is None:
        length = seq_len

    trans_mat, obs_mat, init_dist = params.trans_mat, params.obs_mat, params.init_dist

    trans_mat = jnp.array(trans_mat)
    obs_mat = jnp.array(obs_mat)
    init_dist = jnp.array(init_dist)

    n_states, n_obs = obs_mat.shape

    beta_t = jnp.ones((n_states,))

    def scan_fn(beta_prev, t):
        beta_t = jnp.where(t > length,
                           jnp.zeros_like(beta_prev),
                           normalize((beta_prev * obs_mat[:, obs_seq[-t + 1]] * trans_mat).sum(axis=1))[0])
        return beta_t, beta_t

    ts = jnp.arange(2, seq_len + 1)
    _, beta_hist = lax.scan(scan_fn, beta_t, ts)

    beta_hist = jnp.flip(jnp.vstack([beta_t.reshape(1, n_states), beta_hist]), axis=0)

    return beta_hist


def hmm_forwards_backwards_jax(params, obs_seq, length=None):
    '''
    Computes, for each time step, the marginal conditional probability that the Hidden Markov Model was
    in each possible state given the observations that were made at each time step, i.e.
    P(z[i] | x[0], ..., x[num_steps - 1]) for all i from 0 to num_steps - 1
    Parameters
    ----------
    params : HMMJax
        Hidden Markov Model
    obs_seq: array(seq_len)
        History of observed states
    Returns
    -------
    * array(seq_len, n_states)
        Alpha values
    * array(seq_len, n_states)
        Beta values
    * array(seq_len, n_states)
        Marginal conditional probability
    * float
        The loglikelihood giving log(p(x|model))
    '''
    seq_len = len(obs_seq)
    if length is None:
        length = seq_len

    def gamma_t(t):
        gamma_t = jnp.where(t < length,
                            alpha[t] * beta[t - length],
                            jnp.zeros((n_states,)))
        return gamma_t

    ll, alpha = hmm_forwards_jax(params, obs_seq, length)
    n_states = alpha.shape[1]

    beta = hmm_backwards_jax(params, obs_seq, length)

    ts = jnp.arange(seq_len)
    gamma = vmap(gamma_t, (0))(ts)
    # gamma = alpha * jnp.roll(beta, -seq_len + length, axis=0) #: Alternative
    gamma = vmap(lambda x: normalize(x)[0])(gamma)
    return alpha, beta, gamma, ll


Below are my contributions


In [3]:
@flax.struct.dataclass
class HMMWithActionJax:
    trans_mat: jnp.array  # A : (n_actions, n_states, n_states)
    obs_mat: jnp.array  # B : (n_states, n_obs)
    init_dist: jnp.array  # pi : (n_states)

In [312]:
# Naive (un-vectorized) version
@partial(jax.jit, static_argnums=(1))
def fixed_lag_smoother(params, win_len, alpha_win, obs_seq_win, obs, act=None):
    '''
    Description...

    Parameters
    ----------
    params      : HMMWithActionJax
        Hidden Markov Model (with action-dependent transition)
    win_len     : int
        Desired window length (>= 2)
    alpha_win   : array
        Alpha values for the most recent win_len steps, excluding current step
    obs_seq_win : array
        Observations for the most recent win_len steps, excluding current step
    obs         : int
        New observation for the current step
    act         : array
        (optional) Actions for the most recent win_len steps, including current step
    Returns
    -------
    * array(win_len, n_states)
        Updated alpha values
    * array(win_len)
        Updated observations for the past d steps
    * array(win_len, n_states)
        Smoothed posteriors for the past d steps
    * float
        The loglikelihood of the past d steps
    '''
    if len(alpha_win.shape) < 2:
        alpha_win = jnp.expand_dims(alpha_win, axis=0)
    curr_len = alpha_win.shape[0]
    win_len = min(win_len, curr_len+1)
    assert win_len >= 2, "Must keep a window of length at least 2."

    trans_mat, obs_mat = params.trans_mat, params.obs_mat
    n_states, n_obs = obs_mat.shape
    
    # If trans_mat is independent of action, adjust shape
    if len(trans_mat.shape) < 3:
        trans_mat = jnp.expand_dims(trans_mat, axis=0)
        act = None
    if act is None:
        act = jnp.zeros(shape=(curr_len+1,), dtype=jnp.int8)

    # Shift window forward by 1
    if curr_len == win_len:
        alpha_win = alpha_win[1:]
        obs_seq_win = obs_seq_win[1:]
    new_alpha, _ = normalize(
        obs_mat[:, obs] * (alpha_win[-1][:, None] * trans_mat[act[-1]]).sum(axis=0)
    )
    alpha_win = jnp.concatenate((alpha_win, new_alpha[None, :]), axis=0)
    obs_seq_win = jnp.append(obs_seq_win, obs)

    # Smooth backwards inside the window
    beta_win = jnp.ones(shape=(win_len, n_states))
    gamma_win = jnp.array(alpha_win)
    for t in range(win_len-2, -1, -1):
        new_beta, _ = normalize(
            (jnp.multiply(beta_win[t+1,:], obs_mat[:, obs_seq_win[t+1]]) *
             trans_mat[act[t]]).sum(axis=1)
        )
        beta_win = beta_win.at[t, :].set(new_beta)

        new_gamma, _ = normalize(
            jnp.multiply(alpha_win[t, :], beta_win[t, :])
        )
        gamma_win = gamma_win.at[t, :].set(new_gamma)
    # print(alpha_win, '\n')
    # print(beta_win, '\n')
    return alpha_win, obs_seq_win, beta_win, gamma_win

In [ ]:
# Vectorized version
def fixed_lag_smoother_vectorized(params, win_len, alpha_win, beta_win, obs_seq_win, obs, act=None):
    '''
    Description...

    Parameters
    ----------
    params      : HMMWithActionJax
        Hidden Markov Model (with action-dependent transition)
    win_len     : int
        Desired window length (>= 2)
    alpha_win   : array
        Alpha values for the most recent win_len steps, excluding current step
    beta_win   : array
        Beta values for the most recent win_len steps, excluding current step
    obs_seq_win : array
        Observations for the most recent win_len steps, excluding current step
    obs         : int
        New observation for the current step
    act         : array
        (optional) Actions for the most recent win_len steps, including current step
    Returns
    -------
    * array(win_len, n_states)
        Updated alpha values
    * array(win_len)
        Updated observations for the past d steps
    * array(win_len, n_states)
        Smoothed posteriors for the past d steps
    * float
        The loglikelihood of the past d steps
    '''
    if len(alpha_win.shape) < 2:
        alpha_win = jnp.expand_dims(alpha_win, axis=0)
    curr_len = alpha_win.shape[0]
    win_len = min(win_len, curr_len+1)
    assert win_len >= 2, "Must keep a window of length at least 2."

    trans_mat, obs_mat = params.trans_mat, params.obs_mat
    n_states, n_obs = obs_mat.shape
    
    # If trans_mat is independent of action, adjust shape
    if len(trans_mat.shape) < 3:
        trans_mat = jnp.expand_dims(trans_mat, axis=0)
        act = None
    if act is None:
        act = jnp.zeros(shape=(curr_len+1,), dtype=jnp.int8)

    # Shift window forward by 1
    if curr_len == win_len:
        alpha_win = alpha_win[1:]
        obs_seq_win = obs_seq_win[1:]
    new_alpha, _ = normalize(
        obs_mat[:, obs] * (alpha_win[-1][:, None] * trans_mat[act[-1]]).sum(axis=0)
    )
    alpha_win = jnp.concatenate((alpha_win, new_alpha[None, :]), axis=0)
    obs_seq_win = jnp.append(obs_seq_win, obs)

    # Smooth backwards inside the window
    beta_win = jnp.ones(shape=(win_len, n_states))
    gamma_win = jnp.array(alpha_win)
    for t in range(win_len-2, -1, -1):
        new_beta, _ = normalize(
            (jnp.multiply(beta_win[t+1,:], obs_mat[:, obs_seq_win[t+1]]) *
             trans_mat[act[t]]).sum(axis=1)
        )
        beta_win = beta_win.at[t, :].set(new_beta)

        new_gamma, _ = normalize(
            jnp.multiply(alpha_win[t, :], beta_win[t, :])
        )
        gamma_win = gamma_win.at[t, :].set(new_gamma)
    return alpha_win, obs_seq_win, beta_win, gamma_win

In [307]:
# Umbrella-rain toy example (AI, A Modern Approach)
data = jnp.array([0, 0, 1, 1, 1])
transmat = jnp.array([[0.7, 0.3], [0.3, 0.7]])
obsmat = jnp.array([[0.9, 0.1], [0.2, 0.8]])
prior = jnp.array([0.5, 0.5])

hmm = HMMJax(trans_mat=transmat, obs_mat=obsmat, init_dist=prior)

In [315]:
# Larger-scale example for time experiments
key = PRNGKey(0)
data = jax.random.choice(key, 2, shape=(10000,))

In [316]:
def get_fixed_lag_smoother_result(params, win_len, data, prior, act=None):
    assert data.size > 2, "Complete observation set must be of size at least 2"
    alpha, _ = normalize(jnp.multiply(prior, obsmat[:, data[0]]))
    obs_seq = jnp.array([data[0]])
    for obs in data[1:]:
        alpha, obs_seq, beta, gamma = fixed_lag_smoother(hmm, win_len, alpha, obs_seq, obs)
    return alpha, beta, gamma

In [317]:
alpha1, beta1, gamma1 = get_fixed_lag_smoother_result(hmm, 5, data, prior);
print(beta1)

[[0.57458866 0.4254113 ]
 [0.32730448 0.6726955 ]
 [0.36952144 0.63047856]
 [0.6272727  0.37272727]
 [1.         1.        ]]


In [311]:
alpha1, beta1, gamma1 = get_fixed_lag_smoother_result(hmm, 5, data, prior);

[[0.34444442 0.65555555]
 [1.         1.        ]] 

[[0.36952144 0.6304786 ]
 [0.6272727  0.37272727]
 [1.         1.        ]] 

[[0.35911217 0.64088786]
 [0.58110833 0.4188917 ]
 [0.34444442 0.65555555]
 [1.         1.        ]] 

[[0.36098886 0.63901114]
 [0.5900324  0.4099676 ]
 [0.36952144 0.6304786 ]
 [0.6272727  0.37272727]
 [1.         1.        ]] 

[[0.5864123  0.41358766]
 [0.35911217 0.64088786]
 [0.58110833 0.4188917 ]
 [0.34444442 0.65555555]
 [1.         1.        ]] 

[[0.35756814 0.64243186]
 [0.57354707 0.42645293]
 [0.32465208 0.67534786]
 [0.34444442 0.65555555]
 [1.         1.        ]] 

[[0.57458866 0.42541134]
 [0.32730445 0.6726955 ]
 [0.36952144 0.6304786 ]
 [0.6272727  0.37272727]
 [1.         1.        ]] 

[[0.3261829  0.6738171 ]
 [0.35911217 0.64088786]
 [0.58110833 0.4188917 ]
 [0.34444442 0.65555555]
 [1.         1.        ]] 

[[0.36098886 0.63901114]
 [0.5900324  0.4099676 ]
 [0.36952144 0.6304786 ]
 [0.6272727  0.37272727]
 [1.         1.        ]] 

In [255]:
%%time
alpha, beta, gamma, _ = hmm_forwards_backwards_jax(hmm, data)

CPU times: user 710 ms, sys: 8.28 ms, total: 718 ms
Wall time: 735 ms


In [ ]:
key = PRNGKey(0)
num_states = 3
num_obs = 5
num_timesteps = 15

data = jax.random.choice(key, num_obs, (num_timesteps,))
key, _ = split(key)

transmat = jax.random.uniform(key, shape=(num_states, num_states))
transmat, _ = normalize(transmat, axis=1)
print(transmat)
key, _ = split(key)

obsmat = jax.random.uniform(key, shape=(num_states, num_obs))
obsmat, _ = normalize(obsmat, axis=1)
print(obsmat)
key, _ = split(key)

prior = jnp.array([0.33, 0.33, 0.34])